In [33]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Leitura de instâncias

In [34]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias geradas"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1
    
    # Leitura inicial das demandas
    for i in range(periods):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)
    
    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Reading transfer costs directly from the document as a matrix
    transfer_cost_matrix = []
    transfer_cost_line = demand_start_line + periods

    # Read the matrix of transfer costs line by line
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()
        if line:
            # Split the line into individual cost values and convert them to float
            row = [float(value) for value in line.split()]
            transfer_cost_matrix.append(row)
        transfer_cost_line += 1

    # Convert to a numpy array (optional, if you want to work with numpy for matrix operations)
    transfer_costs = np.array(transfer_cost_matrix)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}


In [35]:
# Exemplo de uso
file_path = '../instancias/multi_plant_instances/AAA02_12_20_120.dat'
data = read_dat_file(file_path)
display(data)

{'items': 120,
 'periods': 12,
 'num_plants': 20,
 'capacities': array([[34346, 34346, 34346, 34346, 34346, 34346, 34346, 34346, 34346,
         34346, 34346, 34346],
        [33508, 33508, 33508, 33508, 33508, 33508, 33508, 33508, 33508,
         33508, 33508, 33508],
        [35664, 35664, 35664, 35664, 35664, 35664, 35664, 35664, 35664,
         35664, 35664, 35664],
        [36606, 36606, 36606, 36606, 36606, 36606, 36606, 36606, 36606,
         36606, 36606, 36606],
        [34893, 34893, 34893, 34893, 34893, 34893, 34893, 34893, 34893,
         34893, 34893, 34893],
        [33659, 33659, 33659, 33659, 33659, 33659, 33659, 33659, 33659,
         33659, 33659, 33659],
        [34582, 34582, 34582, 34582, 34582, 34582, 34582, 34582, 34582,
         34582, 34582, 34582],
        [36234, 36234, 36234, 36234, 36234, 36234, 36234, 36234, 36234,
         36234, 36234, 36234],
        [34942, 34942, 34942, 34942, 34942, 34942, 34942, 34942, 34942,
         34942, 34942, 34942],
        [

# Modelagem

In [36]:
m = gp.Model('Lot-sizing Sambasivan and Yahya')

## Conjuntos

In [37]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

## Parâmetros

In [38]:
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

## Variáveis de decisão

In [39]:
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, lb=0, ub=1, vtype=GRB.CONTINUOUS, name='Z')

## Função objetivo

In [40]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

## Restrições

In [41]:
# Balanço de estoque (revisar comportamento)
# Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
             name='restricao_balanco_estoque')
# Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
             name='restricao_balanco_estoque');

In [42]:
# Restrição que obriga setup (validar o range do r)
m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
             , name='restricao_setup');

In [43]:
# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

In [44]:
m.update()
print(m)
print(m.Fingerprint)

<gurobi.Model Continuous instance Lot-sizing Sambasivan and Yahya: 57840 constrs, 662400 vars, Parameter changes: Username=(user-defined)>
682455576


# Resolução (relax-and-fix)

In [45]:
m.setParam('OutputFlag', 1)  # Desativa a exibição de logs

In [46]:
# Parâmetros
w = 3  # Número de períodos na janela de resolução
y = 1  # Número de períodos em overlap

K = np.ceil((len(T) - w) / (w - y)) + 1  # Número total de iterações
m.Params.timelimit = 1800 / K  # Tempo limite por subproblema (s)
runtime = 0

# Auxiliares
k, to, tf = 1, 0, w
N = T[-1]  # Último período do horizonte de planejamento

# Relaxar domínio das variáveis binárias (Z_ijt)
# for key in Z.keys():
#     Z[key].setAttr("VType", GRB.CONTINUOUS)
#     Z[key].setAttr("LB", 0)
#     Z[key].setAttr("UB", 1)
# m.update()

# print('k:', k, 'to:', to, 'tf:', tf)

# Main loop
while tf <= N:
    # Variáveis na janela tornam-se binárias
    for i in I:
        for j in J:
            for t in range(to, tf):
                Z[i, j, t].setAttr("VType", GRB.BINARY)
    m.update()

    # Resolução do subprolema
    m.optimize()
    if m.Status == GRB.INFEASIBLE:
        break
    # Fixação das soluções obtidas considerando overlap
    for i in I:
        for j in J:
            for t in range(to, tf - y):
                Z[i, j, t].setAttr("LB", Z[i, j, t].X)  # Lower bound fixado para a solução encontrada
                Z[i, j, t].setAttr("UB", Z[i, j, t].X)  # Upper bound fixado para a solução encontrada
    m.update()

    # Atualização dos auxiliares
    k += 1
    to = tf - y
    tf = min(tf + w - y, N + 1)

    # print('k:', k, 'to:', to, 'tf:', tf)

# Última iteração
if m.Status != GRB.INFEASIBLE:
    # Variáveis na janela tornam-se binárias
    for i in I:
        for j in J:
            for t in range(to, tf):
                Z[i, j, t].setAttr("VType", GRB.BINARY)
    m.update()

    # Última resolução
    m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 57840 rows, 662400 columns and 1293600 nonzeros
Model fingerprint: 0x5feca490
Variable types: 655200 continuous, 7200 integer (7200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e-01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 21600 rows and 50400 columns
Presolve time: 2.79s
Presolved: 36240 rows, 612000 columns, 1228800 nonzeros
Variable types: 604800 continuous, 7200 integer (7200 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.24s

Barrier statistics:
 AA' NZ     : 3.288e+05
 Facto

In [51]:
m.SolCount

3

In [47]:
# Número de períodos da instância
len(T)

12

In [48]:
# Número de fábricas
len(J)

20

In [49]:
# Número de produtos
len(I)

120

In [50]:
# Valor da função objetivo da melhor solução encontrada (upper bound da minimização)
m.ObjVal

6758802.6803979855